### Web Scraping Exercise - Fake Jobs

In this exercise, you'll practice using BeautifulSoup to parse the content of a web page. The page that you'll be scraping, https://realpython.github.io/fake-jobs/, contains job listings. Your job is to extract the data on each job and convert into a pandas DataFrame.

##### Question 1

Start by performing a GET request on the url above and convert the response into a BeautifulSoup object.  
a. Use the .find method to find the tag containing the first job title ("Senior Python Developer"). Hint: can you find a tag type and/or a class that could be helpful for extracting this information? Extract the text from this title.  
b. Now, use what you did for the first title, but extract the job title for all jobs on this page. Store the results in a list.  
c. Finally, extract the companies, locations, and posting dates for each job. For example, the first job has a company of "Payne, Roberts and Davis", a location of "Stewartbury, AA", and a posting date of "2021-04-08". Ensure that the text that you extract is clean, meaning no extra spaces or other characters at the beginning or end.  
d. Take the lists that you have created and combine them into a pandas DataFrame. 

In [156]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import re

In [157]:
# Send get request to the fake jobs URL
URL = 'https://realpython.github.io/fake-jobs/'
response = requests.get(URL)

In [158]:
# Convert response to BeautifulSoup object
soup = BeautifulSoup(response.text)

In [159]:
# Extracting the text from the first job title (navigating to the first card, then finding the text whose tag is h2)
soup.find('div', attrs={'class' : 'card'}).h2.text

'Senior Python Developer'

In [160]:
# Iterating through all cards (jobs) on the page
jobs = soup.findAll('div', attrs={'class' : 'card'})

# Extracting the job title for each job
titles_text = [x.h2.text.strip() for x in jobs]

# Extracting the companies for each job
companies_text = [x.h3.text.strip() for x in jobs]

# Extracting the locations for each job
locations_text = [x.p.text.strip() for x in jobs]

# Extracting the posting date for each job
posting_date_text = [x.time.text.strip() for x in jobs]

In [166]:
jobs_df = pd.DataFrame({'Job_Title':titles_text, 'Company':companies_text, 'Location':locations_text, 'Posting_Date':posting_date_text})
jobs_df

,Job_Title,Company,Location,Posting_Date
0,Senior Python Developer,"Payne, Roberts and Davis","Stewartbury, AA",2021-04-08
1,Energy engineer,Vasquez-Davidson,"Christopherville, AA",2021-04-08
2,Legal executive,"Jackson, Chambers and Levy","Port Ericaburgh, AA",2021-04-08
3,Fitness centre manager,Savage-Bradley,"East Seanview, AP",2021-04-08
4,Product manager,Ramirez Inc,"North Jamieview, AP",2021-04-08
...,...,...,...,...
95,Museum/gallery exhibitions officer,"Nguyen, Yoder and Petty","Lake Abigail, AE",2021-04-08
96,"Radiographer, diagnostic",Holder LLC,"Jacobshire, AP",2021-04-08
97,Database administrator,Yates-Ferguson,"Port Susan, AE",2021-04-08
98,Furniture designer,Ortega-Lawrence,"North Tiffany, AA",2021-04-08


##### Question 2

Next, add a column that contains the url for the "Apply" button. Try this in two ways.   
    a. First, use the BeautifulSoup find_all method to extract the urls.  
    b. Next, get those same urls in a different way. Examine the urls and see if you can spot the pattern of how they are constructed. Then, build the url using the elements you have already extracted. Ensure that the urls that you created match those that you extracted using BeautifulSoup. Warning: You will need to do some string cleaning and prep in constructing the urls this way. For example, look carefully at the urls for the "Software Engineer (Python)" job and the "Scientist, research (maths)" job.

In [162]:
# Adding a column that contains the url for the "Apply" button via find_all
jobs_df['URL'] = [x.findAll('a')[1].get('href') for x in jobs]

In [163]:
pd.set_option('display.max_colwidth', None)
jobs_df[jobs_df['Job_Title'].isin(['Software Engineer (Python)','Scientist, research (maths)'])]

,Job_Title,Company,Location,Posting_Date,URL
10,Software Engineer (Python),Garcia PLC,"Ericberg, AE",2021-04-08,https://realpython.github.io/fake-jobs/jobs/software-engineer-python-10.html
22,"Scientist, research (maths)","Manning, Welch and Herring","Laurenland, AE",2021-04-08,https://realpython.github.io/fake-jobs/jobs/scientist-research-maths-22.html


In [164]:
# Adding a column that contains the url for the "Apply" button via pattern matching

# First, remove all parentheses and commas from job titles
titles_stripped = [re.sub(r'[(),]', '', title) for title in titles_text]
# Next, replace any slashes with spaces
titles_stripped = [re.sub(r'[\/]', ' ', title) for title in titles_stripped]

# Create a list containing the number to append at the end of each URL
indices = range(len(titles_stripped))

# Define the base URL and extension
base_URL = 'https://realpython.github.io/fake-jobs/jobs/'
extension = '.html'

# Build up the URLs using list comprehension, starting with base URL, adding job title with spaces replaced by hyphens followed by index, and ending with extension
built_URLs = [base_URL
              + job.lower().replace(' ','-')+'-'+str(index)
              + extension 
              for job, index in zip(titles_stripped,indices)]

In [165]:
assert jobs_df['URL'].to_list() == built_URLs

##### Question 3

Finally, we want to get the job description text for each job.  
    a. Start by looking at the page for the first job, https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html. Using BeautifulSoup, extract the job description paragraph.  
    b. We want to be able to do this for all pages. Write a function which takes as input a url and returns the description text on that page. For example, if you input "https://realpython.github.io/fake-jobs/jobs/television-floor-manager-8.html" into your function, it should return the string "At be than always different American address. Former claim chance prevent why measure too. Almost before some military outside baby interview. Face top individual win suddenly. Parent do ten after those scientist. Medical effort assume teacher wall. Significant his himself clearly very. Expert stop area along individual. Three own bank recognize special good along.".  
    c. Use the [.apply method](https://pandas.pydata.org/docs/reference/api/pandas.Series.apply.html) on the url column you created above to retrieve the description text for all of the jobs.

In [167]:
# Extracting job description for first job

# Send get request to the job application URL
URL = 'https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html'
response = requests.get(URL)

# Convert response to BeautifulSoup object
soup = BeautifulSoup(response.text)

# Extracting the text from the content class
soup.find('div', attrs={'class' : 'content'}).p.text

'Professional asset web application environmentally friendly detail-oriented asset. Coordinate educational dashboard agile employ growth opportunity. Company programs CSS explore role. Html educational grit web application. Oversea SCRUM talented support. Web Application fast-growing communities inclusive programs job CSS. Css discussions growth opportunity explore open-minded oversee. Css Python environmentally friendly collaborate inclusive role. Django no experience oversee dashboard environmentally friendly willing to learn programs. Programs open-minded programs asset.'

In [150]:
# Defining a function which takes as input a url and returns the description text on that page
def get_job_description(job_url):
    response = requests.get(job_url)
    soup = BeautifulSoup(response.text)
    return soup.find('div', attrs={'class' : 'content'}).p.text

get_job_description('https://realpython.github.io/fake-jobs/jobs/television-floor-manager-8.html')

'At be than always different American address. Former claim chance prevent why measure too. Almost before some military outside baby interview. Face top individual win suddenly. Parent do ten after those scientist. Medical effort assume teacher wall. Significant his himself clearly very. Expert stop area along individual. Three own bank recognize special good along.'

In [151]:
job_descriptions = jobs_df['URL'].apply(get_job_description)

In [155]:
job_descriptions[0:3]

0    Professional asset web application environmentally friendly detail-oriented asset. Coordinate educational dashboard agile employ growth opportunity. Company programs CSS explore role. Html educational grit web application. Oversea SCRUM talented support. Web Application fast-growing communities inclusive programs job CSS. Css discussions growth opportunity explore open-minded oversee. Css Python environmentally friendly collaborate inclusive role. Django no experience oversee dashboard environmentally friendly willing to learn programs. Programs open-minded programs asset.
1                                                                                                                                                                                                                                                                                                                                                                                                                              